In [1]:
import sys
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch

args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'custom' # data
args.root_path = '/root/study/ChainMT/Informer/processed_data/2017/' # root path of data file
#args.data_path = 'betti.csv' # data file
#args.data_path = 'betti_der.csv' # data file
#args.data_path = 'fl.csv' # data file
args.data_path = 'base.csv'
args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'price' # target feature in S or MS task
args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 2 # input sequence length of Informer encoder
args.label_len = 1 # start token length of Informer decoder
args.pred_len = 10 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 7 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder

#args.batch_size = 32 
args.batch_size = 7
args.learning_rate = 0.0001

#args.loss = 'mse'
args.loss = 'rmse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'


In [2]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

def select_data(dataset_model):
    # Set augments by using data name
    if(dataset_model=='base'):
        data_parser = {
            'custom':{'data':'base.csv','T':'price','M':[2,2,2],'S':[1,1,1],'MS':[2,2,1]}, # base
        }
    elif(dataset_model=='betti'):
        data_parser = {
            'custom':{'data':'betti.csv','T':'price','M':[22,22,22],'S':[1,1,1],'MS':[22,22,1]}, # betti
        }
    elif(dataset_model=='betti_der'):
        data_parser = {
            'custom':{'data':'betti_der.csv','T':'price','M':[22,22,22],'S':[1,1,1],'MS':[22,22,1]}, # betti derivate
        }
    elif(dataset_model=='fl'):
        data_parser = {
            'custom':{'data':'fl.csv','T':'price','M':[22,22,22],'S':[1,1,1],'MS':[22,22,1]}, # fl
        }

    if args.data in data_parser.keys():
        data_info = data_parser[args.data]
        args.data_path = data_info['data']
        args.target = data_info['T']
        args.enc_in, args.dec_in, args.c_out = data_info[args.features]

    args.detail_freq = args.freq
    args.freq = args.freq[-1:]
    print('Args in experiment:')
    print(args)

In [19]:
import numpy as np
Exp = Exp_Informer
window_arr = [3,5,7]
horizon_arr = [1, 2, 5, 7, 10, 15, 20, 25, 30]
dataset_model_array = ["base","betti", "betti_der","fl"]
rmse_total = np.zeros([4, 3, 9])
i = 0
for dataset_model in dataset_model_array:
    args.data_path = dataset_model + '.csv'
    select_data(dataset_model)
    j = 0
    for args.seq_len in window_arr:
        n = 0
        for args.pred_len in horizon_arr:
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model, args.data, args.features, 
                            args.seq_len, args.label_len, args.pred_len,
                            args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.des, ii)

                # set experiments
                exp = Exp(args)
                # train
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                # test
                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                rmse_total[i][j][n] = exp.test(setting)
                

                torch.cuda.empty_cache()
                
            n = n + 1
        j = j + 1
    i = i + 1

Args in experiment:
{'model': 'informer', 'data': 'custom', 'root_path': '/root/study/ChainMT/Informer/processed_data/2017/', 'data_path': 'base.csv', 'features': 'MS', 'target': 'price', 'freq': 'h', 'checkpoints': './informer_checkpoints', 'seq_len': 3, 'label_len': 1, 'pred_len': 5, 'enc_in': 2, 'dec_in': 2, 'c_out': 1, 'factor': 5, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'batch_size': 7, 'learning_rate': 0.0001, 'loss': 'rmse', 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 6, 'patience': 3, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 'h'}
Use CPU
>>>>>>>start training : informer_custom_ftMS_sl3_ll1_pl1_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 252
val 37
test 73
Epoch: 1 cost time: 3.4023268222808

In [22]:
np.save("/root/study/ChainMT/Informer/2017_bitcoin_informer_rmse_total.npy", rmse_total)